## 355. Design Twitter
- Description:
  <blockquote>
    Design a simplified version of Twitter where users can post tweets, follow/unfollow another user, and is able to see the `10` most recent tweets in the user's news feed.
     
    Implement the `Twitter` class:
     
    - `Twitter()` Initializes your twitter object.
    - `void postTweet(int userId, int tweetId)` Composes a new tweet with ID `tweetId` by the user `userId`. Each call to this function will be made with a unique `tweetId`.
    - `List<Integer> getNewsFeed(int userId)` Retrieves the `10` most recent tweet IDs in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user themself. Tweets must be **ordered from most recent to least recent**.
    - `void follow(int followerId, int followeeId)` The user with ID `followerId` started following the user with ID `followeeId`.
    - `void unfollow(int followerId, int followeeId)` The user with ID `followerId` started unfollowing the user with ID `followeeId`.
     
    **Example 1:**
    **Input**
    ["Twitter", "postTweet", "getNewsFeed", "follow", "postTweet", "getNewsFeed", "unfollow", "getNewsFeed"]
    `[],  [1, 5], [1], [1, 2], [2, 6], [1], [1, 2], [1]`
    **Output**
    [null, null, [5], null, null, [6, 5], null, [5]]
     
    **Explanation**
    Twitter twitter = new Twitter();
    twitter.postTweet(1, 5); // User 1 posts a new tweet (id = 5).
    twitter.getNewsFeed(1);  // User 1's news feed should return a list with 1 tweet id -> [5]. return [5]
    twitter.follow(1, 2);    // User 1 follows user 2.
    twitter.postTweet(2, 6); // User 2 posts a new tweet (id = 6).
    twitter.getNewsFeed(1);  // User 1's news feed should return a list with 2 tweet ids -> [6, 5]. Tweet id 6 should precede tweet id 5 because it is posted after tweet id 5.
    twitter.unfollow(1, 2);  // User 1 unfollows user 2.
    twitter.getNewsFeed(1);  // User 1's news feed should return a list with 1 tweet id -> [5], since user 1 is no longer following user 2.
     
    **Constraints:**
     
    - `1 <= userId, followerId, followeeId <= 500`
    - `0 <= tweetId <= 104`
    - All the tweets have **unique** IDs.
    - At most `3 * 104` calls will be made to `postTweet`, `getNewsFeed`, `follow`, and `unfollow`.
    - A user cannot follow himself.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/design-twitter/description/)

- Topics: Max Heap, Sorting

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, (Optimized), Heap-Based Merge

Time Complexity:

    Initialize heap: push latest tweet from each of F users → O(F log F)
    Then, pop up to K=10 times; each pop/push is O(log F)

➡️ Total: O(F log F + K log F) = O(F log F)
Since K = 10 is constant, this simplifies to O(F log F)

    With F ≤ 500, this is ~500 × log₂(500) ≈ 500 × 9 = ~4500 operations, much better than sorting 30k tweets.

Space Complexity:

    Heap stores at most F elements → O(F)
    No need to collect all tweets

➡️ Extra space per call: O(F)

In [ ]:
class Twitter:

    def __init__(self):
        # userId -> set of followeeIds
        self.following = defaultdict(set)
        # userId -> list of (timestamp, tweetId)
        self.tweets = defaultdict(list)
        self.timestamp = 0

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweets[userId].append((self.timestamp, tweetId))
        self.timestamp += 1

    def getNewsFeed(self, userId: int) -> List[int]:
        followees = self.following[userId].union({userId})
        
        heap = []
        # Push the latest tweet from each followee (if exists)
        for uid in followees:
            if self.tweets[uid]:
                # (negative time for max-heap), tweetId, userId, index
                time, tid = self.tweets[uid][-1]
                heapq.heappush(heap, (-time, tid, uid, len(self.tweets[uid]) - 1))
        
        feed = []
        while heap and len(feed) < 10:
            neg_time, tid, uid, idx = heapq.heappop(heap)
            feed.append(tid)
            # Push next most recent tweet from same user (if any)
            if idx > 0:
                new_idx = idx - 1
                time, tid = self.tweets[uid][new_idx]
                heapq.heappush(heap, (-time, tid, uid, new_idx))
        
        return feed

    def follow(self, followerId: int, followeeId: int) -> None:
        self.following[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followerId in self.following:
            # safe remove as it does not raise an error if followeeId does not exist in the given set
            self.following[followerId].discard(followeeId)


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)

### Solution 2, Collect + Sort All Tweets

Time Complexity:

    Collect tweets: O(T)
    Sort them: O(T log T)
    Slice top 10: O(1)

➡️ Total: O(T log T)

    In worst case, a user follows 500 people, each with ~60 tweets (since total calls ≤ 30k), so T ≈ 30,000 → sorting 30k items per feed request could be heavy if called frequently.

Space Complexity:

    Store all tweets: O(T) (same as overall storage)
    Temporary list for sorting: O(T)

➡️ Extra space per call: O(T)

In [ ]:
class Twitter:

    def __init__(self):
        # userId -> set of followeeIds
        self.following = defaultdict(set)
        # userId -> list of (timestamp, tweetId)
        self.tweets = defaultdict(list)
        self.timestamp = 0

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweets[userId].append((self.timestamp, tweetId))
        self.timestamp += 1

    def getNewsFeed(self, userId: int) -> List[int]:
        all_tweets = []
        
        # Gather all relevant users: self + followees
        users = {userId}
        users.update(self.following[userId])
        
        # Gather all tweets from all the relevant users
        for user in users:
            if user in self.tweets:
                all_tweets.extend(self.tweets[user])
        
        all_tweets.sort(key=lambda x: x[0], reverse=True)
        
        return [tid for _, tid in all_tweets[:10]]

    def follow(self, followerId: int, followeeId: int) -> None:
        self.following[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followerId in self.following:
            # safe remove as it does not raise an error if followeeId does not exist in the given set
            self.following[followerId].discard(followeeId)


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)

Extra sol slow

In [ ]:
# Solution techniques are make a userid to tweet deque dict, append new tweets to the left of deque with a timer / counter variable to mark insertion order, to retrieve top 10 tweets
# do a union of all followee ids and gather tweets from all and then merge using heap and get top 10

# Time complexity : O() Space complexity : O()


class Twitter:

    def __init__(self):
        self.timer = itertools.count(step=-1)
        self.tweets = collections.defaultdict(collections.deque)
        self.followees = collections.defaultdict(set)

    def postTweet(self, userId, tweetId):
        self.tweets[userId].appendleft((next(self.timer), tweetId))

    def getNewsFeed(self, userId):
        tweets = heapq.merge(*(self.tweets[u] for u in self.followees[userId] | {userId}))
        return [t for _, t in itertools.islice(tweets, 10)]

    def follow(self, followerId, followeeId):
        self.followees[followerId].add(followeeId)

    def unfollow(self, followerId, followeeId):
        self.followees[followerId].discard(followeeId)